In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
from datetime import date
# Install the missing module
%pip install plotly
import plotly.graph_objects as go

# Mock implementation of Weigfun if the module is not available
def Weigfun(T_lag):
	# Replace this with the actual implementation if available
	return np.ones(int(T_lag)) / T_lag

Note: you may need to restart the kernel to use updated packages.


`````{admonition} Interactive Python Page
:class: tip, dropdown

The code on this page can be used interactively: click {fa}`rocket` --> {guilabel}`Live Code` in the top right corner, then wait until the message {guilabel}`Python interaction ready!` appears.

When this page is activated:
- Several packages will be imported automatically
- Code cells will **not** be executed automatically (you do it!)

````{admonition} Which packages are imported when this page is activated?
:class: note, dropdown
```
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
from datetime import date
from plot_hydrograph import plot_hydrograph, interactive_plot
from Weigfun import Weigfun```
````
`````

# Models Exercise 2: Lumped Conceptual Model

In this exercise you will create a lumped conceptual model. In the figure below a schematization of the conceptual model is given.


![image.png](figures/lumped_model.png)

## Part 1

Write down all the different parts of the water balance with the correct units. Make use of the lecture slides and the equations and statements below.   

Water can evaporate from the interception reservoir at the potential rate E$_p$. 

Water can spill from the reservoir when the level S$_i$ reaches I$_{max}$ (mm). 

P$_e$ is partitioned into P$_i$ and P$_f$ according to C$_r$, calculated as: 

$Cr=(S_u/S_{umax})^{\beta}$

Resulting for the fast storage: $Q_{uf} = C_r*P_e$; 

From which it can flow out: $Q_f=K_f*S_f$; 

For the underground storage holds: $Q_{iu} =(1-C_r)*P_e$. 

From here it can evaporate following: $E_a=S_u/(S_{umax}*C_e)*E_p$; 

Or it can percolate further into the ground: $Q_{us} =P_{max}*(S_u/S_{umax})$; 

From this storage it can flow out: $Q_s=K_s*S_s$. 

Q$_f$ and Q$_s$ are summed and routed through the triangular transfer function with base T$_{lag}$

## Part 2 Create HBV model

In this part you will code the lumped model, which is also called the HBV-model. Below the function ```HBVMod``` is given, but as you can see, part of the function is unfinished. Only edit the part below the statement ```UPDATE THE PART BELOW``` and above ```END```. 

After you finish this function, you can plot the hydrograph and see if the function is correct. The plot is interactive, use the sliders to change a parameter and see what their effect is.

In [2]:
def HBVMod(Par,forcing,S_in, hydrograph):
    # HBVpareto Calculates values of 3 objective functions for HBV model

    I_max = Par[0]
    Ce = Par[1]
    Su_max = Par[2]
    beta = Par[3]
    P_max = Par[4]
    T_lag = Par[5]
    Kf = Par[6]
    Ks = Par[7]
    

    Prec = forcing['P'].values
    Qo = forcing['Q'].values
    Etp = forcing['PE'].values


    t_max = len(Prec)
    
    # Allocate Si, Su, Sf, Ss, Ei_dt, Ea_dt, Q_tot_dt
    
    Si = np.zeros(t_max)
    Su = np.zeros(t_max)
    Sf = np.zeros(t_max)
    Ss = np.zeros(t_max)
    Ei_dt = np.zeros(t_max)
    Ea_dt =  np.zeros(t_max)
    Q_tot_dt = np.zeros(t_max)
    Qs = np.zeros(t_max)
    Qf = np.zeros(t_max)
    
    # Initialize Si, Su, Sf, Ss
    Si[0] = S_in[0]
    Su[0] = S_in[1]
    Sf[0] = S_in[2]
    Ss[0] = S_in[3]

    dt = 1

    for i in range(0, t_max):
        P_dt = Prec[i] * dt
        Ep_dt = Etp[i] * dt
        
        # Interception Reservoir
        if P_dt > 0:
            Si[i] = Si[i] + P_dt 
            Pe_dt = np.maximum(0, (Si[i] - I_max) / dt)
            Si[i] = Si[i] - Pe_dt
            Ei_dt[i] = 0
        else:
        # Evaporation only when there is no rainfall
            Pe_dt = np.maximum(0, (Si[i] - I_max) / dt) #is zero, because of no rainfall
            Ei_dt[i] = np.minimum(Ep_dt, Si[i] / dt)
            Si[i] = Si[i] - Pe_dt - Ei_dt[i]
        
        if i < t_max-1:
            Si[i+1] = Si[i]
        
        
        # Split Pe into Unsaturated Reservoir and Preferential Reservoir
        if Pe_dt > 0:
            Cr = (Su[i] / Su_max) ** beta
            Qiu_dt = (1 - Cr) * Pe_dt # flux from Ir to Ur
            Su[i] = Su[i] + Qiu_dt
            Quf_dt = Cr * Pe_dt #flux from Su to Sf
        else:
            Quf_dt = 0
        
        # Transpiration
        Ep_dt = max(0, Ep_dt - Ei_dt[i])
        Ea_dt[i] = Ep_dt * (Su[i] / (Su_max * Ce))
        Ea_dt[i] = min(Su[i] / dt, Ea_dt[i])
        Su[i] = Su[i] - Ea_dt[i]
        
        # Percolation
        Qus_dt = P_max * (Su[i] / Su_max) * dt  # Flux from Su to Ss
        Su[i] = Su[i] - Qus_dt
        
        if i < t_max - 1:
            Su[i+1] = Su[i]
        
        # Fast Reservoir
        Sf[i] = Sf[i] + Quf_dt
        Qf_dt = dt * Kf * Sf[i]
        Sf[i] = Sf[i] - Qf_dt
        if i < t_max-1:
            Sf[i+1] = Sf[i]
        
        # Slow Reservoir
        Ss[i] = Ss[i] + Qus_dt
        Qs_dt = dt * Ks * Ss[i]
        Ss[i] = Ss[i] - Qs_dt
        if i < t_max-1:
            Ss[i+1] = Ss[i]
        
        Q_tot_dt[i] = Qs_dt + Qf_dt
        Qs[i] = Qs_dt 
        Qf[i] = Qf_dt 


    # Check Water Balance
    Sf = Si[-1] + Ss[-1] + Sf[-1] + Su[-1]  # final storage
    S_in = sum(S_in)  # initial storage
    WB = sum(Prec) - sum(Ei_dt) - sum(Ea_dt) - sum(Q_tot_dt) - Sf + S_in
    # print(WB)
    # Offset Q

    Weigths = Weigfun(T_lag)
    
    Qm = np.convolve(Q_tot_dt, Weigths)
    Qm = Qm[0:t_max]
    forcing['Qm'] = Qm
   
    if hydrograph == 'TRUE':
    ## Plot
    # hour=1:t_max\
        fig, ax = plt.subplots(figsize=(12,8))
        forcing['Q'].plot(label='Obserbed', ax=ax)
        forcing['Qm'].plot(label='Model',  ax=ax)
        ax.legend()
        

    return Qm

    

Before we can compute the discharge based on your model, we need to import the forcing data from the text file into a dataframe. 

In [4]:
import xarray as xr
# --- 1. Load Precipitation Data (NetCDF) ---
def load_precipitation(filepath):
    ds = xr.open_dataset(filepath)
    df = ds.to_dataframe().reset_index()
    df = df[df['bnds'] == 0]  # Filter to keep valid bounds
    df.index = pd.to_datetime(df['time'])
    df.index = df.index.normalize()  # Set index to Timestamp (no time component)
    df = df.drop(columns=['time', 'bnds'])
    prec = df['pr'] * 86400  # Convert from m/s to mm/day
    prec[prec < 0] = np.nan  # Handle negative values
    return prec

prec = load_precipitation("OBS6_ERA5_reanaly_1_day_pr_2000-2018_newbasin.nc")

# --- 2. Load Potential Evapotranspiration (NetCDF) ---
def load_evapotranspiration(filepath):
    ds = xr.open_dataset(filepath)
    df = ds.to_dataframe().reset_index()
    df.index = pd.to_datetime(df['time'])
    df.index = df.index.normalize()
    ep = df['evspsblpot'] * 86400  # Convert from mm/s to mm/day
    ep[ep < 0] = np.nan
    return ep
EP = load_evapotranspiration("Derived_Makkink_evspsblpot_newbasin.nc")

# --- 3. Load Discharge Data (CSV) ---
def load_discharge(filepath, basin_area):
    df = pd.read_csv(filepath, delim_whitespace=True, skiprows=1, header=None, encoding='latin1')
    df = df[[0, 3]]  # Keep only date and Q columns
    df.columns = ['Date', 'Q']
    df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
    df.set_index('Date', inplace=True)
    df['Q'] = (df['Q'] / basin_area) * 1e3  # Convert to mm/day
    df['Q'][df['Q'] < 0] = np.nan
    return df['Q']

A = 17002.553 *10**6 #m^2
dfQ = load_discharge("Upstream_Q.txt", A)

# --- 4. Align Time Series (2000-2017) ---
def align_data(prec, EP, Q, start_year=2000):
    # Filter by date range
    prec = prec[(prec.index.year >= start_year)]
    EP = EP[(EP.index.year >= start_year)]
    Q = Q[(Q.index.year >= start_year)]
    # Merge into a single DataFrame
    aligned_data = pd.concat([prec, EP, Q], axis=1, join='inner')
    aligned_data.columns = ['P', 'PE', 'Q']
    return aligned_data

forcing= align_data(prec, EP, dfQ)
# Drop rows where any column has a NaN value
forcing = forcing.dropna()
print(forcing.head())

                   P        PE         Q
2000-01-01  0.004437  1.156963  0.000000
2000-01-02  0.000000  1.175265  0.082691
2000-01-03  0.020500  1.067080  0.079740
2000-01-04  0.000000  1.165309  0.077725
2000-01-05  0.000000  1.169341  0.073602


C:\Users\margh\AppData\Local\Temp\ipykernel_91848\955227733.py:29: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  df = pd.read_csv(filepath, delim_whitespace=True, skiprows=1, header=None, encoding='latin1')
C:\Users\margh\AppData\Local\Temp\ipykernel_91848\955227733.py:35: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this 

To create an interactive plot, the sets with minimum and maximum parameters should be given. If you want you can change these values. 

If you run the cell below, an interactive plot will be generated. In this plot you can change the values of the different parameters, play with the values and see their effect on the hydrograph and the NSE value. How many times did you change the parameters to get a reasonable NSE value?

**Before you close this exercise: the function ```HBVMod``` you fixed above is required as an input for the first eWatercycle exercise. If you close your browser you loose what you have made. So: before you close this window, copy - paste your ```HBVMod``` function into a local text file on your computer!**

In [5]:
params = {
    'I_max': {'min': 0, 'max': 10},
    'Ce': {'min': 0.1, 'max': 1},
    'Su_max': {'min': 40, 'max': 800},
    'beta': {'min': 0.5, 'max': 5},
    'P_max': {'min': 0.001, 'max': 0.3},
    'T_lag': {'min': 1, 'max': 10},
    'Kf': {'min': 0.01, 'max': 0.1},
    'Ks': {'min': 0.0001, 'max': 0.01},
}


In [12]:
import pandas as pd
import plotly.graph_objects as go

def plot_interactive_hydrograph(df, observed_col='Observed', simulated_col='Simulated', time_col='Date'):
    """
    Plot interactive hydrograph with Plotly.

    Parameters:
    - df: DataFrame with datetime and discharge data
    - observed_col: name of the observed discharge column
    - simulated_col: name of the simulated/model discharge column
    - time_col: name of the datetime column
    """
    fig = go.Figure()

    if observed_col in df.columns:
        fig.add_trace(go.Scatter(
            x = df[time_col] if time_col else df.index,
            y=df[observed_col],
            mode='lines',
            name='Observed',
            line=dict(color='blue')
        ))

    if simulated_col in df.columns:
        fig.add_trace(go.Scatter(
            x = df[time_col] if time_col else df.index,
            y=df[simulated_col],
            mode='lines',
            name='Simulated',
            line=dict(color='red', dash='dash')
        ))

    fig.update_layout(
        title='Interactive Hydrograph',
        xaxis_title='Date',
        yaxis_title='Discharge [m³/s]',
        template='plotly_white',
        hovermode='x unified',
        legend=dict(x=0, y=1),
        height=500
    )

    fig.show()


In [13]:
from ipywidgets import interact, FloatSlider

def interactive_model_plot(I_max, Ce, Su_max, beta, P_max, T_lag, Kf, Ks):
    # Update parameters
    Par = [I_max, Ce, Su_max, beta, P_max, T_lag, Kf, Ks]
    S_in = [0, 0, 0, 0]  # Initial storage values for Si, Su, Sf, Ss
    # Recalculate the modeled discharge
    Qm = HBVMod(Par=Par, forcing=forcing, S_in=S_in, hydrograph='FALSE')
    
    # Update the forcing DataFrame with the new modeled discharge
    forcing['Qm'] = Qm
    
    # Plot the interactive hydrograph
    plot_interactive_hydrograph(forcing, observed_col='Q', simulated_col='Qm', time_col=None)

# Create interactive sliders for each parameter
interact(
    interactive_model_plot,
    I_max=FloatSlider(min=params['I_max']['min'], max=params['I_max']['max'], step=0.1, value=params['I_max']['max']),
    Ce=FloatSlider(min=params['Ce']['min'], max=params['Ce']['max'], step=0.01, value=params['Ce']['max']),
    Su_max=FloatSlider(min=params['Su_max']['min'], max=params['Su_max']['max'], step=10, value=params['Su_max']['max']),
    beta=FloatSlider(min=params['beta']['min'], max=params['beta']['max'], step=0.1, value=params['beta']['max']),
    P_max=FloatSlider(min=params['P_max']['min'], max=params['P_max']['max'], step=0.001, value=params['P_max']['max']),
    T_lag=FloatSlider(min=params['T_lag']['min'], max=params['T_lag']['max'], step=1, value=params['T_lag']['max']),
    Kf=FloatSlider(min=params['Kf']['min'], max=params['Kf']['max'], step=0.001, value=params['Kf']['max']),
    Ks=FloatSlider(min=params['Ks']['min'], max=params['Ks']['max'], step=0.0001, value=params['Ks']['max'])
)

interactive(children=(FloatSlider(value=10.0, description='I_max', max=10.0), FloatSlider(value=1.0, descripti…

<function __main__.interactive_model_plot(I_max, Ce, Su_max, beta, P_max, T_lag, Kf, Ks)>